#### Imports

In [1]:
import PyPDF2
import csv
import re
import json
import os

#### Scrape key to facility codes
- Very inclusive info about keys, many things not keys included

In [2]:
pdfFileObj = open('2007/directory_2007.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
print("Number of pages:", pdfReader.numPages)

Number of pages: 680


In [3]:
def clean_key_page(raw_abbr_key):
    raw_abbr_key = raw_abbr_key.replace('\n', ' ').replace('(', ' ').replace(')', ' ').split(' ')
    raw_abbr_key[:] = [l for l in raw_abbr_key if (l.strip()!= '')]
    i = 0
    while i < len(raw_abbr_key):
        #print(raw_abbr_key[i])
        if raw_abbr_key[i].isupper():
            abbr_key.append(raw_abbr_key[i])
        i += 1

In [4]:
abbr_key = []

In [5]:
raw_abbr_key1 = pdfReader.getPage(4).extractText()
clean_key_page(raw_abbr_key1)
raw_abbr_key2 = pdfReader.getPage(5).extractText()
clean_key_page(raw_abbr_key2)

In [6]:
abbr_key

['TO',
 'FACILITY',
 'CODES*',
 'SA',
 'MH',
 'MH-SA',
 'GH',
 'TX',
 'DT',
 'MM',
 'DM',
 'BU',
 'HH',
 'HI',
 'OP',
 'PH',
 'RS',
 'RL',
 'AD',
 'CO',
 'HV',
 'HIV/AIDS',
 'GL',
 'SE',
 'PW',
 'WN',
 'MN',
 'BC',
 'DU',
 'DUI/DWI',
 'CJ',
 'SF',
 'MD',
 'MC',
 'SI',
 'PI',
 'MI',
 'VA,',
 'TRICARE',
 'AR',
 'SS',
 'PA',
 'AH',
 'ASL',
 'SP',
 'TO',
 'SPECIAL',
 'LANGUAGE',
 'SERVICES',
 'NOTE:',
 'N2',
 'N15',
 'N24',
 'N33',
 'N3',
 'N18',
 'N25',
 'N34',
 'N5',
 'N19',
 'N26',
 'N35',
 'N6',
 'N20',
 'N28',
 'N37',
 'N8',
 'N21',
 'N29',
 'N40',
 'N13',
 'N22',
 'N30',
 'N41',
 'N14',
 'N23',
 'N32',
 'F1',
 'F28',
 'F52',
 'F74',
 'F2',
 'F30',
 'F54',
 'F75',
 'F3',
 'F31',
 'F55',
 'F77',
 'F4',
 'F33',
 'F56',
 'SP',
 'F5',
 'F34',
 'F57',
 'F78',
 'F8',
 'F35',
 'F58',
 'F79',
 'F9',
 'F36',
 'F59',
 'F81',
 'F10',
 'F37',
 'F60',
 'F83',
 'F11',
 'F38',
 'F61',
 'F86',
 'F15',
 'F39',
 'F62',
 'F87',
 'F16',
 'F40',
 'F63',
 'F88',
 'F17',
 'F41',
 'F64',
 'F89',
 'F18',
 'F4

#### Scraper test on a page of data
- data starts on page 15 (getPage(14))

In [161]:
def split_data(rawdata):
    '''
    Roughly split data, returns list of list.
    Inputs:
        rawdata_from_page(list): list of items on the page that were seperated by /n
    '''
    #split erroneously connected data
    data = []
    temp = []
    i = 0
    while i < len(rawdata) - 1:
        #if i is the last line of keys, i+1 is facility name, i-1 is the phone number or other line of keys
        if rawdata[i].isupper() and (rawdata[i].split()[0] in abbr_key) and \
        (not rawdata[i+1].split()[0] in abbr_key): #and (rawdata[i-1][0] == '(' or rawdata[i-1].isupper()): 
            temp.append(rawdata[i])
            data.append(temp)
            temp = []
        else:
            temp.append(rawdata[i])
        i += 1
    temp.append(rawdata[i])
    data.append(temp)
    return data
    
    
def clean_facility_name(dcs):
    '''
    Move and link wrongly splitted data, changes in place
    '''
    i = 0
    while i < len(dcs):
        cn = dcs[i][0]
        if cn.isupper():
            del dcs[i][0]
        else:
            if cn[0].isupper() and cn[1].isnumeric():
                upper_end = re.search(r'\d+', cn).end()
                if cn[:upper_end - 1] in abbr_key:
                    dcs[i - 1].append(cn[:upper_end - 1])
                dcs[i][0] = cn[upper_end - 1:]
                
            elif cn[:2].isupper():
                if 'Inc' in cn:
                    if cn[:2] in abbr_key:
                        dcs[i - 1].append(cn[:2]) 
                    dcs[i][0] = cn[2:]
                    
                else:
                    upper_end = re.search(r'[A-Z ]+', cn).end()
                    if cn[:upper_end - 1] in abbr_key:
                        dcs[i - 1].append(cn[:upper_end - 1]) 
                    dcs[i][0] = cn[upper_end - 1:]
        i += 1

In [162]:
def split_center_street(cn_sa, page):
    '''
    Split a string containing center name and street address into two.
    Inputs:
        cn_sa(str)
    Returns: a list
    '''
    #Exception to split at number :
        #When it ends with "P.O. Box",
        #When the number being found is at the very front (wrongly included page number or 
        #center name starts with number)
        
    #If the number is at the end(suite xxx or building xxx) lets include the word in front of it)
    #print(cn_sa)
    if cn_sa[0].isnumeric():
        cn_sa[0].replace(' For Code Definitions - See KEY on Page v. and cover flap', '')
    
            
    m = re.finditer(r"\d+", cn_sa)

    for num in m:
        span = num.span()
        if span[0] != 0:    
            if cn_sa[span[0] - 9:span[0] - 1] == 'P.O. Box':
                return ([cn_sa[:span[0] - 10], cn_sa[span[0] - 9:]])
            if span[1] != len(cn_sa):
                return ([cn_sa[:span[0] - 1], cn_sa[span[0]:]])
            if span[1] == len(cn_sa):
                return ([' '.join(cn_sa.split()[:-2]), ' '.join(cn_sa.split()[-2:])])
    return [cn_sa, '']


def write_data(year, page, dc, writer):
    '''
    Writes data in dc, returns appending data if any
    '''
    i = 0
    data_holder = [year, page]
    #Center name, street address
    cn_sa = ''
    while (',' not in dc[i]) or not dc[i][-5:].isnumeric(): #(len(dc[i]) < 2) or 
        cn_sa += ' ' + dc[i]
        i += 1
    data_holder += split_center_street(cn_sa.strip(), page)

    #City, State, Postal_code
    city, state_postcode = dc[i].split(',')
    state, postcode = state_postcode.split()
    data_holder += [city, state, postcode]
    i += 1
    #phone + keys
    phone = dc[i]
    i += 1
    while not dc[i][0:2].isupper():
        phone += ' ' + dc[i]
        i += 1
    if phone[-1].isupper():
        key_start = re.search(r'[A-Z]{2}', phone).start()
        data_holder.append(phone[:key_start])
        data_holder.append(phone[key_start:] + ' ' + ' '.join(dc[i:]))
    else:
        key_start = len(phone)
        data_holder.append(phone)
        data_holder.append(' '.join(dc[i:]))
        
    if not data_holder[-1].isupper(): #if the last line of keys also contains another center
        keys_tail = data_holder[-1]
        tail_start = re.search(r'[A-Z][a-z]', keys_tail).start()
        data_holder[-1] = keys_tail[: tail_start]
        return keys_tail[tail_start:]
        
    #write data
    #print(data_holder)
    writer.writerow(data_holder)

In [173]:
def run(year, start_page, end_page = 0):
    
    #read file
    dir_filename = '{0}/directory_{0}.pdf'.format(year)
    pdfFileObj = open(dir_filename, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    
    #pages to scrape
    if end_page == 0:
        end_page = pdfReader.numPages - 1
    
    #open/create csv
    save_as_filename = '{0}/{0}.csv'.format(year)
    if os.path.isfile(save_as_filename):
        with open(save_as_filename, 'a', newline='') as file:
            writer = csv.writer(file)
    else:
        with open(save_as_filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Year", "Page", "Center_name", "Street_address", "City", \
                             "State", "Postal_code", "Phone", "Keys"])

    for i in range(start_page, end_page + 1):
        page = pdfReader.getPage(i).extractText()
        rawdata = page.split('\n')
        
        
        rawdata[:] = [l.replace('˜', ' ').replace('  ', ' ').replace('  ', ' ').strip() for l in rawdata \
                      if not (l.strip() == '' or l == 'KEY')]
        #print(rawdata)
        if (len(rawdata) > 3):
            data = split_data(rawdata)
            clean_facility_name(data)
            with open(save_as_filename, 'a', newline='') as file:
                writer = csv.writer(file)
                for dc in data:
                    write_data(year, i + 1, dc, writer)

In [177]:
run(2007, 365) #355, 364, 365, 394

IndexError: list index out of range